In [1]:
import init
import mnistdata as mylib
import numpy as np
from matplotlib import pyplot
import matplotlib as mpl
import matplotlib.pyplot as plt
import math as math
import preprocessing as pp
import knnfeatures as kf
import datetime
import sys


def show(image,image2='None'):
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image,cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    if(image2!='None'):
        ax = fig.add_subplot(2,2,1)
        imgplot = ax.imshow(image2,cmap=mpl.cm.Greys)
        imgplot.set_interpolation('nearest')
        ax.xaxis.set_ticks_position('top')
        ax.yaxis.set_ticks_position('left')
    
    pyplot.show()

def euclideanDistance(x1, x2):
    dis = sum(pow(x1-x2,2))
    return np.sqrt(dis) 
    
def oneNormDistance(x1, x2):
    dis = sum(abs(x1-x2))
    return dis

def infNormDistance(x1,x2):
    dis = np.zeros(x1.size)
    for i in range(x1.size):
        dis[i] = max(x1[i],x2[i])
    return(sum(dis))

def mahalonobisDist(x1,x2,invS):
    diff = np.matrix(x1-x2)
    print(diff)
    dis = (diff*invS)*diff.T
    return dis    

def getFeatureVector(Image):
    #pre-processed Image
    ppImage = pp.boundarysquare(pp.thresholding(Image,0))
    #ppImage = pp.thresholding(Image,150)
    #ppImage = pp.thin1(ppImage,1)
    #ppImage = pp.thin1(ppImage,2)
    featureVector = []
    #feature 1: Number of on pixels
    featureVector.append(kf.averagemass(ppImage))
    #feature 2,3: Number of Left/Right pixels
    lrmass = kf.averagelrmass(ppImage)
    #print(lrmass)
    featureVector.append(lrmass[0])
    featureVector.append(lrmass[1])
    #feature 4,5: Number of Top/Bottom pixels
    tbmass = kf.averagetbmass(ppImage)
    featureVector.append(tbmass[0])
    featureVector.append(tbmass[1]) 
    #feature 6: avgHorizontalStroke
    featureVector.append(kf.avgHorizontalStroke(ppImage))
    #feature 7: avgVerticalStroke
    featureVector.append(kf.avgVerticalStroke(ppImage))
    #feature 8: The number transitions from black to white or vice versa when moving accross the image 
    featureVector.append(kf.transitions(ppImage))
    #feature 9,10,11,12: No of pixels touching the boundary of the cropped image
    featureVector.append(kf.topBoundaryTouch(ppImage))
    featureVector.append(kf.bottomBoundaryTouch(ppImage))
    featureVector.append(kf.leftBoundaryTouch(ppImage))
    featureVector.append(kf.rightBoundaryTouch(ppImage))
    #feature 13: Aspect Ratio
    featureVector.append(kf.aspectRatio(ppImage))
    #feature 14: avgDistanceFromImageCenter
    featureVector.append(kf.avgDistanceFromImageCenter(ppImage))
    #feature 15: How symmetric is the picture wth respect to the vertical axis
    featureVector.append(kf.ySymmetric(ppImage))
    #featureVector.append(kf.ySymmetric2(ppImage))    
    #feature 15: How symmetric is the picture wth respect to the horizontal axis
    featureVector.append(kf.xSymmetric(ppImage))
    
    #featureVector.append(kf.lrRatio(ppImage))
    #featureVector.append(kf.tbRatio(ppImage))
    return np.array(featureVector)

def getAllFeatureVectors(images,N=10):
    image = []
    dataFeatures = []
    i=0
    while((i<images.shape[0])and(i<N)):
        dataFeatures.append(getFeatureVector(images[i]))
        i+=1
    return np.array(dataFeatures)


def getNeighbors(X, Z, xt, k, dist=euclideanDistance):
    distances = []
    for x1 in X:
        distance = dist(x1,xt)
        distances.append(distance)
    sortedDistances = distances[:]
    sortedDistances.sort()
    neighbors = []
    for d in sortedDistances:
        if(len(neighbors)<k):
            for index in range(len(distances)):
                if(d == distances[index] and len(neighbors)<k):
                    neighbor = (X[index],Z[index],d)
                    neighbors.append(neighbor)
    return neighbors

def getResponse(neighbors, c=10):
    response = -1
    classVotes = [0] * c
    for neighbor in neighbors:
        classVotes[int(neighbor[1])]=classVotes[int(neighbor[1])]+1
    response = np.argmax(classVotes)
    return response  

def getAccuracy(YT, ZT):
    success = 0
    size = len(YT)
    for i in range(size):
        if(YT[i] == ZT[i]):
            success+=1
    accuracy = (success/float(size))*100
    print(success)
    return accuracy

def predict(X, Z, XT, k):
    Y=[]
    i = 0
    for xt in XT:
        #print(datetime.datetime.now())                
        #print(i)
        neighbors = getNeighbors(X, Z, xt, k)
        Y.append(getResponse(neighbors))
        i+=1
    return Y

def meanVector(dataFeatures):
    mean = []
    dim = np.shape(dataFeatures)
    for j in range(dim[1]):
        mean.append(np.mean(dataFeatures[:,j]))
    return np.array(mean)
       
def minuMean(dataFeatures):
    mean = meanVector(dataFeatures)
    dim = np.shape(dataFeatures)
    f = []
    for i in range(dim[0]):
        f.append(dataFeatures[i,:]-mean)
    return np.array(f)

def scaledFeaturesVectors(dataFeatures,eigVectors,eigValues,mean):
    sc = []
    dim = np.shape(dataFeatures)
    c = np.matmul(eigValues,np.transpose(eigVectors))
    for i in range(dim[0]):
        m = dataFeatures[i,:]-mean
        sc.append(np.matmul(c,m))
    return(np.array(sc))

def eigAnalysis(dataFeatures_train):
    mean = meanVector(dataFeatures_train)
    #dataFeatures_train = minuMean(dataFeatures_train)
    cov = np.cov(dataFeatures_train.T)
    eigValues,eigVectors = np.linalg.eig(cov)
    for i in range(eigValues.size):
        if eigValues[i]<0:
            #print(eigValues[i])
            eigValues[i] = 0.000000001
    eigValuesD = np.diag(eigValues)
    eigValuesD = np.linalg.inv(eigValuesD)
    eigValuesD = np.sqrt(eigValuesD)
    return eigValuesD,eigVectors,mean

print("start : "+ str(datetime.datetime.now()))
Ntr = 2000
Nte = 1000

lbl_tr, img_tr = mylib.read("training","../data")
dataFeatures_train = getAllFeatureVectors(img_tr,Ntr)
eigValues,eigVectors,mean = eigAnalysis(dataFeatures_train)
dataFeatures_train = scaledFeaturesVectors(dataFeatures_train,eigVectors,eigValues,mean)

lbl_te, img_te = mylib.read("testing","../data")
dataFeatures_test = getAllFeatureVectors(img_te,Nte)
dataFeatures_test = scaledFeaturesVectors(dataFeatures_test,eigVectors,eigValues,mean)

k = 15
YT = predict(dataFeatures_train[0:Ntr], lbl_tr[0:Ntr], dataFeatures_test[0:Nte], k)
accuracy = getAccuracy(YT, lbl_te[0:Nte])
#print(np.array(YT))
#print(lbl_te[0:Nte])
print('Accuracy: ' + repr(accuracy))
print("end : "+ str(datetime.datetime.now()))

ImportError: No module named 'init'